# Spectral Entropy

In [ ]:
import librosa
import os
import numpy as np
import scipy
from scipy import signal
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import sys
import pathlib
import glob
import seaborn as sns
plt.rcParams.update({'font.size': 18})

## Load Data

In [ ]:
base_path = "/home/ubuntu/"

biden_ai = base_path + 'data/world-leaders-dataset/biden_ai.wav'
biden_real = base_path + 'data/world-leaders-dataset/biden_original.wav'

timit_real = base_path + 'data/timit-test/example_real_resampled.wav'
timit_og_real = base_path + 'data/timit-test/example_real_original.wav'
timit_fake = base_path + 'data/timit-test/example_fake_resampled.wav'

## Extract F0

Reference: https://librosa.org/doc/main/generated/librosa.pyin.html

- __f0:__ time series of fundamental frequencies in Hertz

- __voiced_flag:__ time series containing boolean flags indicating whether a frame is voiced or not.

- __voiced_prob:__ time series containing the probability that a frame is voiced.

The min and max frequencies passed as params while extracting F0 recommended by librosa are 'C2' ~65 hz and 'C7' ~2093 hz.

The voiced speech of a typical adult male will have a fundamental frequency from 85 to 155 Hz, and that of a typical adult female from 165 to 255 Hz. [Voice frequency](https://en.wikipedia.org/wiki/Voice_frequency). 

Function returns `nan` values whenever there is no voiced segment


In [ ]:
def extract_f0_sequence(audio_file, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7')):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #use pYIN to extract f0 and voiced segment flag + probaility
    f0, voiced_flag, voiced_probs = librosa.pyin(audio, fmin=fmin, 
                                                 fmax=fmax, fill_na=np.nan)
    
    #generate ndarray of times (in seconds) corresponding to each frame of X
    times = librosa.times_like(f0)
    
    return f0, times, voiced_flag, voiced_probs

In [ ]:
def plot_f0_spectrogram(audio_file, f0_sequence, f0_times):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #generate spectrogram
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)
    
    #plot spectrogram from stft
    fig, ax = plt.subplots(figsize=(14, 5))
    img = librosa.display.specshow(D, x_axis='time', y_axis='log', ax=ax)
    ax.set(title='pYIN fundamental frequency estimation')
    fig.colorbar(img, ax=ax, format="%+2.f dB")
    
    #overlay f0 sequence
    ax.plot(f0_times, f0_sequence, label='f0', color='cyan', linewidth=3)
    ax.legend(loc='upper right')

In [ ]:
f0_jbreal, times_jb_real, _, _ = extract_f0_sequence(biden_real) 
f0_jbfake, times_jb_fake, _, _ = extract_f0_sequence(biden_ai) 

In [ ]:
plot_f0_spectrogram(biden_real, f0_jbreal, times_jb_real)

In [ ]:
plot_f0_spectrogram(biden_ai, f0_jbfake, times_jb_fake)

In [ ]:
f0_timit_real, times_timit_real, _, _ = extract_f0_sequence(timit_real)
f0_timit_og_real, times_timit_og_real, _, _ = extract_f0_sequence(timit_og_real) 
f0_timit_fake, times_timit_fake, _, _ = extract_f0_sequence(timit_fake)

In [ ]:
plot_f0_spectrogram(timit_real, f0_timit_real, times_timit_real)

In [ ]:
plot_f0_spectrogram(timit_og_real, f0_timit_og_real, times_timit_og_real)

In [ ]:
plot_f0_spectrogram(timit_fake, f0_timit_fake, times_timit_fake)

In [ ]:
audio_ts, sr = librosa.load(timit_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-6])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(timit_fake)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

## Spectral Entropy:

References:
- https://kilthub.cmu.edu/articles/thesis/Audio_Deepfake_Detection_Based_on_Differences_in_Human_and_Machine_Generated_Speech/21842454

- https://dsp.stackexchange.com/questions/23689/what-is-spectral-entropy

![image-2.png](attachment:image-2.png)


__Notes:__

- Typically PSD is computed using a periodogram. Is it OK to use an STFT for the audio signal?
- stft output is `freq_bins * frames`

In [ ]:
audio_ts, sr = librosa.load(biden_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(biden_ai)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
def compute_spectral_entropy(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    normalized_psd = psd/np.sum(psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_psd)
    
    return spectral_entropy

In [ ]:
# this function needs to be reviewed
def compute_spectral_entropy_f0(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    f0_sequence, _, _, _ = extract_f0_sequence(audio_file)
    
    f0_psd = np.zeros(len(f0_sequence))
    
    for i,f0 in enumerate(f0_sequence):
        if np.isnan(f0):
            f0_psd[i]=0
        else:
            difference_array = np.abs(freqs-f0)
            index = difference_array.argmin()
            f0_psd[i] = psd[index]
    
    normalized_f0_psd = f0_psd/np.sum(f0_psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_f0_psd)
    
    return spectral_entropy

In [ ]:
compute_spectral_entropy(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy_f0(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy(timit_real), compute_spectral_entropy(timit_fake)

In [ ]:
compute_spectral_entropy_f0(timit_real), compute_spectral_entropy(timit_fake)

## Testing for LibriSpeech and generated versions:

In [ ]:
LJ_original = '/Users/gautham/datasets/wavefake_data/LJSpeech_1.1/wavs'

LJ_fbmelgan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_full_band_melgan'

LJ_parallel_wavegan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_parallel_wavegan'

In [ ]:
original_files = []
for dirpath,_,filenames in os.walk(LJ_original):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            original_files.append(os.path.abspath(os.path.join(dirpath, file)))
original_files.sort()
print(len(original_files))

In [ ]:
melgan_files = []
for dirpath,_,filenames in os.walk(LJ_fbmelgan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            melgan_files.append(os.path.abspath(os.path.join(dirpath, file)))
melgan_files.sort()
print(len(melgan_files))

In [ ]:
wavegan_files = []
for dirpath,_,filenames in os.walk(LJ_parallel_wavegan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            wavegan_files.append(os.path.abspath(os.path.join(dirpath, file)))
wavegan_files.sort()
print(len(wavegan_files))

In [ ]:
compute_spectral_entropy(original_files[0]), compute_spectral_entropy(melgan_files[0]), compute_spectral_entropy(wavegan_files[0])

In [ ]:
compute_spectral_entropy_f0(original_files[0]), compute_spectral_entropy_f0(melgan_files[0]), compute_spectral_entropy_f0(wavegan_files[0])

In [ ]:
original_files_entropy = [compute_spectral_entropy(file) for file in original_files]
melgan_files_entropy = [compute_spectral_entropy(file) for file in melgan_files]
wavegan_files_entropy = [compute_spectral_entropy(file) for file in wavegan_files]

In [ ]:
plt.figure(figsize=(10, 10), dpi=80)
sns.kdeplot(original_files_entropy, label='original')
sns.kdeplot(melgan_files_entropy, label='melgan')
sns.kdeplot(wavegan_files_entropy, label='wavegan')
plt.legend()
plt.show()

In [ ]:
f0_orig, times_orig, _, _ = extract_f0_sequence(original_files[0]) 
f0_melgan, times_melgan, _, _ = extract_f0_sequence(melgan_files[0])
f0_wavegan, times_wavegan, _, _ = extract_f0_sequence(wavegan_files[0])

In [ ]:
print(original_files[0])
print(melgan_files[0])
print(wavegan_files[0])

In [ ]:
plot_f0_spectrogram(original_files[0], f0_orig, times_orig)

In [ ]:
plot_f0_spectrogram(melgan_files[0], f0_melgan, times_melgan)

In [ ]:
plot_f0_spectrogram(wavegan_files[0], f0_wavegan, times_wavegan)

In [ ]:
audio_ts, sr = librosa.load(original_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(melgan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(wavegan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

# Spectral Entropy

In [ ]:
import librosa
import os
import numpy as np
import scipy
from scipy import signal
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import sys
import pathlib
import glob
import seaborn as sns
plt.rcParams.update({'font.size': 18})

## Load Data

In [ ]:
base_path = "/home/ubuntu/"

biden_ai = base_path + 'data/world-leaders-dataset/biden_ai.wav'
biden_real = base_path + 'data/world-leaders-dataset/biden_original.wav'

timit_real = base_path + 'data/timit-test/example_real_resampled.wav'
timit_og_real = base_path + 'data/timit-test/example_real_original.wav'
timit_fake = base_path + 'data/timit-test/example_fake_resampled.wav'

## Extract F0

Reference: https://librosa.org/doc/main/generated/librosa.pyin.html

- __f0:__ time series of fundamental frequencies in Hertz

- __voiced_flag:__ time series containing boolean flags indicating whether a frame is voiced or not.

- __voiced_prob:__ time series containing the probability that a frame is voiced.

The min and max frequencies passed as params while extracting F0 recommended by librosa are 'C2' ~65 hz and 'C7' ~2093 hz.

The voiced speech of a typical adult male will have a fundamental frequency from 85 to 155 Hz, and that of a typical adult female from 165 to 255 Hz. [Voice frequency](https://en.wikipedia.org/wiki/Voice_frequency). 

Function returns `nan` values whenever there is no voiced segment


In [ ]:
def extract_f0_sequence(audio_file, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7')):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #use pYIN to extract f0 and voiced segment flag + probaility
    f0, voiced_flag, voiced_probs = librosa.pyin(audio, fmin=fmin, 
                                                 fmax=fmax, fill_na=np.nan)
    
    #generate ndarray of times (in seconds) corresponding to each frame of X
    times = librosa.times_like(f0)
    
    return f0, times, voiced_flag, voiced_probs

In [ ]:
def plot_f0_spectrogram(audio_file, f0_sequence, f0_times):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #generate spectrogram
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)
    
    #plot spectrogram from stft
    fig, ax = plt.subplots(figsize=(14, 5))
    img = librosa.display.specshow(D, x_axis='time', y_axis='log', ax=ax)
    ax.set(title='pYIN fundamental frequency estimation')
    fig.colorbar(img, ax=ax, format="%+2.f dB")
    
    #overlay f0 sequence
    ax.plot(f0_times, f0_sequence, label='f0', color='cyan', linewidth=3)
    ax.legend(loc='upper right')

In [ ]:
f0_jbreal, times_jb_real, _, _ = extract_f0_sequence(biden_real) 
f0_jbfake, times_jb_fake, _, _ = extract_f0_sequence(biden_ai) 

In [ ]:
plot_f0_spectrogram(biden_real, f0_jbreal, times_jb_real)

In [ ]:
plot_f0_spectrogram(biden_ai, f0_jbfake, times_jb_fake)

In [ ]:
f0_timit_real, times_timit_real, _, _ = extract_f0_sequence(timit_real)
f0_timit_og_real, times_timit_og_real, _, _ = extract_f0_sequence(timit_og_real) 
f0_timit_fake, times_timit_fake, _, _ = extract_f0_sequence(timit_fake)

In [ ]:
plot_f0_spectrogram(timit_real, f0_timit_real, times_timit_real)

In [ ]:
plot_f0_spectrogram(timit_og_real, f0_timit_og_real, times_timit_og_real)

In [ ]:
plot_f0_spectrogram(timit_fake, f0_timit_fake, times_timit_fake)

In [ ]:
audio_ts, sr = librosa.load(timit_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-6])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(timit_fake)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

## Spectral Entropy:

References:
- https://kilthub.cmu.edu/articles/thesis/Audio_Deepfake_Detection_Based_on_Differences_in_Human_and_Machine_Generated_Speech/21842454

- https://dsp.stackexchange.com/questions/23689/what-is-spectral-entropy

![image-2.png](attachment:image-2.png)


__Notes:__

- Typically PSD is computed using a periodogram. Is it OK to use an STFT for the audio signal?
- stft output is `freq_bins * frames`

In [ ]:
audio_ts, sr = librosa.load(biden_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(biden_ai)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
def compute_spectral_entropy(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    normalized_psd = psd/np.sum(psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_psd)
    
    return spectral_entropy

In [ ]:
# this function needs to be reviewed
def compute_spectral_entropy_f0(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    f0_sequence, _, _, _ = extract_f0_sequence(audio_file)
    
    f0_psd = np.zeros(len(f0_sequence))
    
    for i,f0 in enumerate(f0_sequence):
        if np.isnan(f0):
            f0_psd[i]=0
        else:
            difference_array = np.abs(freqs-f0)
            index = difference_array.argmin()
            f0_psd[i] = psd[index]
    
    normalized_f0_psd = f0_psd/np.sum(f0_psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_f0_psd)
    
    return spectral_entropy

In [ ]:
compute_spectral_entropy(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy_f0(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy(timit_real), compute_spectral_entropy(timit_fake)

In [ ]:
compute_spectral_entropy_f0(timit_real), compute_spectral_entropy(timit_fake)

## Testing for LibriSpeech and generated versions:

In [ ]:
LJ_original = '/Users/gautham/datasets/wavefake_data/LJSpeech_1.1/wavs'

LJ_fbmelgan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_full_band_melgan'

LJ_parallel_wavegan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_parallel_wavegan'

In [ ]:
original_files = []
for dirpath,_,filenames in os.walk(LJ_original):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            original_files.append(os.path.abspath(os.path.join(dirpath, file)))
original_files.sort()
print(len(original_files))

In [ ]:
melgan_files = []
for dirpath,_,filenames in os.walk(LJ_fbmelgan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            melgan_files.append(os.path.abspath(os.path.join(dirpath, file)))
melgan_files.sort()
print(len(melgan_files))

In [ ]:
wavegan_files = []
for dirpath,_,filenames in os.walk(LJ_parallel_wavegan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            wavegan_files.append(os.path.abspath(os.path.join(dirpath, file)))
wavegan_files.sort()
print(len(wavegan_files))

In [ ]:
compute_spectral_entropy(original_files[0]), compute_spectral_entropy(melgan_files[0]), compute_spectral_entropy(wavegan_files[0])

In [ ]:
compute_spectral_entropy_f0(original_files[0]), compute_spectral_entropy_f0(melgan_files[0]), compute_spectral_entropy_f0(wavegan_files[0])

In [ ]:
original_files_entropy = [compute_spectral_entropy(file) for file in original_files]
melgan_files_entropy = [compute_spectral_entropy(file) for file in melgan_files]
wavegan_files_entropy = [compute_spectral_entropy(file) for file in wavegan_files]

In [ ]:
plt.figure(figsize=(10, 10), dpi=80)
sns.kdeplot(original_files_entropy, label='original')
sns.kdeplot(melgan_files_entropy, label='melgan')
sns.kdeplot(wavegan_files_entropy, label='wavegan')
plt.legend()
plt.show()

In [ ]:
f0_orig, times_orig, _, _ = extract_f0_sequence(original_files[0]) 
f0_melgan, times_melgan, _, _ = extract_f0_sequence(melgan_files[0])
f0_wavegan, times_wavegan, _, _ = extract_f0_sequence(wavegan_files[0])

In [ ]:
print(original_files[0])
print(melgan_files[0])
print(wavegan_files[0])

In [ ]:
plot_f0_spectrogram(original_files[0], f0_orig, times_orig)

In [ ]:
plot_f0_spectrogram(melgan_files[0], f0_melgan, times_melgan)

In [ ]:
plot_f0_spectrogram(wavegan_files[0], f0_wavegan, times_wavegan)

In [ ]:
audio_ts, sr = librosa.load(original_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(melgan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(wavegan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

# Spectral Entropy

In [ ]:
import librosa
import os
import numpy as np
import scipy
from scipy import signal
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import sys
import pathlib
import glob
import seaborn as sns
plt.rcParams.update({'font.size': 18})

## Load Data

In [ ]:
base_path = "/home/ubuntu/"

biden_ai = base_path + 'data/world-leaders-dataset/biden_ai.wav'
biden_real = base_path + 'data/world-leaders-dataset/biden_original.wav'

timit_real = base_path + 'data/timit-test/example_real_resampled.wav'
timit_og_real = base_path + 'data/timit-test/example_real_original.wav'
timit_fake = base_path + 'data/timit-test/example_fake_resampled.wav'

## Extract F0

Reference: https://librosa.org/doc/main/generated/librosa.pyin.html

- __f0:__ time series of fundamental frequencies in Hertz

- __voiced_flag:__ time series containing boolean flags indicating whether a frame is voiced or not.

- __voiced_prob:__ time series containing the probability that a frame is voiced.

The min and max frequencies passed as params while extracting F0 recommended by librosa are 'C2' ~65 hz and 'C7' ~2093 hz.

The voiced speech of a typical adult male will have a fundamental frequency from 85 to 155 Hz, and that of a typical adult female from 165 to 255 Hz. [Voice frequency](https://en.wikipedia.org/wiki/Voice_frequency). 

Function returns `nan` values whenever there is no voiced segment


In [ ]:
def extract_f0_sequence(audio_file, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7')):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #use pYIN to extract f0 and voiced segment flag + probaility
    f0, voiced_flag, voiced_probs = librosa.pyin(audio, fmin=fmin, 
                                                 fmax=fmax, fill_na=np.nan)
    
    #generate ndarray of times (in seconds) corresponding to each frame of X
    times = librosa.times_like(f0)
    
    return f0, times, voiced_flag, voiced_probs

In [ ]:
def plot_f0_spectrogram(audio_file, f0_sequence, f0_times):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #generate spectrogram
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)
    
    #plot spectrogram from stft
    fig, ax = plt.subplots(figsize=(14, 5))
    img = librosa.display.specshow(D, x_axis='time', y_axis='log', ax=ax)
    ax.set(title='pYIN fundamental frequency estimation')
    fig.colorbar(img, ax=ax, format="%+2.f dB")
    
    #overlay f0 sequence
    ax.plot(f0_times, f0_sequence, label='f0', color='cyan', linewidth=3)
    ax.legend(loc='upper right')

In [ ]:
f0_jbreal, times_jb_real, _, _ = extract_f0_sequence(biden_real) 
f0_jbfake, times_jb_fake, _, _ = extract_f0_sequence(biden_ai) 

In [ ]:
plot_f0_spectrogram(biden_real, f0_jbreal, times_jb_real)

In [ ]:
plot_f0_spectrogram(biden_ai, f0_jbfake, times_jb_fake)

In [ ]:
f0_timit_real, times_timit_real, _, _ = extract_f0_sequence(timit_real)
f0_timit_og_real, times_timit_og_real, _, _ = extract_f0_sequence(timit_og_real) 
f0_timit_fake, times_timit_fake, _, _ = extract_f0_sequence(timit_fake)

In [ ]:
plot_f0_spectrogram(timit_real, f0_timit_real, times_timit_real)

In [ ]:
plot_f0_spectrogram(timit_og_real, f0_timit_og_real, times_timit_og_real)

In [ ]:
plot_f0_spectrogram(timit_fake, f0_timit_fake, times_timit_fake)

In [ ]:
audio_ts, sr = librosa.load(timit_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-6])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(timit_fake)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

## Spectral Entropy:

References:
- https://kilthub.cmu.edu/articles/thesis/Audio_Deepfake_Detection_Based_on_Differences_in_Human_and_Machine_Generated_Speech/21842454

- https://dsp.stackexchange.com/questions/23689/what-is-spectral-entropy

![image-2.png](attachment:image-2.png)


__Notes:__

- Typically PSD is computed using a periodogram. Is it OK to use an STFT for the audio signal?
- stft output is `freq_bins * frames`

In [ ]:
audio_ts, sr = librosa.load(biden_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(biden_ai)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
def compute_spectral_entropy(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    normalized_psd = psd/np.sum(psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_psd)
    
    return spectral_entropy

In [ ]:
# this function needs to be reviewed
def compute_spectral_entropy_f0(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    f0_sequence, _, _, _ = extract_f0_sequence(audio_file)
    
    f0_psd = np.zeros(len(f0_sequence))
    
    for i,f0 in enumerate(f0_sequence):
        if np.isnan(f0):
            f0_psd[i]=0
        else:
            difference_array = np.abs(freqs-f0)
            index = difference_array.argmin()
            f0_psd[i] = psd[index]
    
    normalized_f0_psd = f0_psd/np.sum(f0_psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_f0_psd)
    
    return spectral_entropy

In [ ]:
compute_spectral_entropy(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy_f0(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy(timit_real), compute_spectral_entropy(timit_fake)

In [ ]:
compute_spectral_entropy_f0(timit_real), compute_spectral_entropy(timit_fake)

## Testing for LibriSpeech and generated versions:

In [ ]:
LJ_original = '/Users/gautham/datasets/wavefake_data/LJSpeech_1.1/wavs'

LJ_fbmelgan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_full_band_melgan'

LJ_parallel_wavegan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_parallel_wavegan'

In [ ]:
original_files = []
for dirpath,_,filenames in os.walk(LJ_original):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            original_files.append(os.path.abspath(os.path.join(dirpath, file)))
original_files.sort()
print(len(original_files))

In [ ]:
melgan_files = []
for dirpath,_,filenames in os.walk(LJ_fbmelgan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            melgan_files.append(os.path.abspath(os.path.join(dirpath, file)))
melgan_files.sort()
print(len(melgan_files))

In [ ]:
wavegan_files = []
for dirpath,_,filenames in os.walk(LJ_parallel_wavegan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            wavegan_files.append(os.path.abspath(os.path.join(dirpath, file)))
wavegan_files.sort()
print(len(wavegan_files))

In [ ]:
compute_spectral_entropy(original_files[0]), compute_spectral_entropy(melgan_files[0]), compute_spectral_entropy(wavegan_files[0])

In [ ]:
compute_spectral_entropy_f0(original_files[0]), compute_spectral_entropy_f0(melgan_files[0]), compute_spectral_entropy_f0(wavegan_files[0])

In [ ]:
original_files_entropy = [compute_spectral_entropy(file) for file in original_files]
melgan_files_entropy = [compute_spectral_entropy(file) for file in melgan_files]
wavegan_files_entropy = [compute_spectral_entropy(file) for file in wavegan_files]

In [ ]:
plt.figure(figsize=(10, 10), dpi=80)
sns.kdeplot(original_files_entropy, label='original')
sns.kdeplot(melgan_files_entropy, label='melgan')
sns.kdeplot(wavegan_files_entropy, label='wavegan')
plt.legend()
plt.show()

In [ ]:
f0_orig, times_orig, _, _ = extract_f0_sequence(original_files[0]) 
f0_melgan, times_melgan, _, _ = extract_f0_sequence(melgan_files[0])
f0_wavegan, times_wavegan, _, _ = extract_f0_sequence(wavegan_files[0])

In [ ]:
print(original_files[0])
print(melgan_files[0])
print(wavegan_files[0])

In [ ]:
plot_f0_spectrogram(original_files[0], f0_orig, times_orig)

In [ ]:
plot_f0_spectrogram(melgan_files[0], f0_melgan, times_melgan)

In [ ]:
plot_f0_spectrogram(wavegan_files[0], f0_wavegan, times_wavegan)

In [ ]:
audio_ts, sr = librosa.load(original_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(melgan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(wavegan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

# Spectral Entropy

In [ ]:
import librosa
import os
import numpy as np
import scipy
from scipy import signal
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import sys
import pathlib
import glob
import seaborn as sns
plt.rcParams.update({'font.size': 18})

## Load Data

In [ ]:
base_path = "/home/ubuntu/"

biden_ai = base_path + 'data/world-leaders-dataset/biden_ai.wav'
biden_real = base_path + 'data/world-leaders-dataset/biden_original.wav'

timit_real = base_path + 'data/timit-test/example_real_resampled.wav'
timit_og_real = base_path + 'data/timit-test/example_real_original.wav'
timit_fake = base_path + 'data/timit-test/example_fake_resampled.wav'

## Extract F0

Reference: https://librosa.org/doc/main/generated/librosa.pyin.html

- __f0:__ time series of fundamental frequencies in Hertz

- __voiced_flag:__ time series containing boolean flags indicating whether a frame is voiced or not.

- __voiced_prob:__ time series containing the probability that a frame is voiced.

The min and max frequencies passed as params while extracting F0 recommended by librosa are 'C2' ~65 hz and 'C7' ~2093 hz.

The voiced speech of a typical adult male will have a fundamental frequency from 85 to 155 Hz, and that of a typical adult female from 165 to 255 Hz. [Voice frequency](https://en.wikipedia.org/wiki/Voice_frequency). 

Function returns `nan` values whenever there is no voiced segment


In [ ]:
def extract_f0_sequence(audio_file, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7')):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #use pYIN to extract f0 and voiced segment flag + probaility
    f0, voiced_flag, voiced_probs = librosa.pyin(audio, fmin=fmin, 
                                                 fmax=fmax, fill_na=np.nan)
    
    #generate ndarray of times (in seconds) corresponding to each frame of X
    times = librosa.times_like(f0)
    
    return f0, times, voiced_flag, voiced_probs

In [ ]:
def plot_f0_spectrogram(audio_file, f0_sequence, f0_times):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #generate spectrogram
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)
    
    #plot spectrogram from stft
    fig, ax = plt.subplots(figsize=(14, 5))
    img = librosa.display.specshow(D, x_axis='time', y_axis='log', ax=ax)
    ax.set(title='pYIN fundamental frequency estimation')
    fig.colorbar(img, ax=ax, format="%+2.f dB")
    
    #overlay f0 sequence
    ax.plot(f0_times, f0_sequence, label='f0', color='cyan', linewidth=3)
    ax.legend(loc='upper right')

In [ ]:
f0_jbreal, times_jb_real, _, _ = extract_f0_sequence(biden_real) 
f0_jbfake, times_jb_fake, _, _ = extract_f0_sequence(biden_ai) 

In [ ]:
plot_f0_spectrogram(biden_real, f0_jbreal, times_jb_real)

In [ ]:
plot_f0_spectrogram(biden_ai, f0_jbfake, times_jb_fake)

In [ ]:
f0_timit_real, times_timit_real, _, _ = extract_f0_sequence(timit_real)
f0_timit_og_real, times_timit_og_real, _, _ = extract_f0_sequence(timit_og_real) 
f0_timit_fake, times_timit_fake, _, _ = extract_f0_sequence(timit_fake)

In [ ]:
plot_f0_spectrogram(timit_real, f0_timit_real, times_timit_real)

In [ ]:
plot_f0_spectrogram(timit_og_real, f0_timit_og_real, times_timit_og_real)

In [ ]:
plot_f0_spectrogram(timit_fake, f0_timit_fake, times_timit_fake)

In [ ]:
audio_ts, sr = librosa.load(timit_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-6])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(timit_fake)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

## Spectral Entropy:

References:
- https://kilthub.cmu.edu/articles/thesis/Audio_Deepfake_Detection_Based_on_Differences_in_Human_and_Machine_Generated_Speech/21842454

- https://dsp.stackexchange.com/questions/23689/what-is-spectral-entropy

![image-2.png](attachment:image-2.png)


__Notes:__

- Typically PSD is computed using a periodogram. Is it OK to use an STFT for the audio signal?
- stft output is `freq_bins * frames`

In [ ]:
audio_ts, sr = librosa.load(biden_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(biden_ai)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
def compute_spectral_entropy(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    normalized_psd = psd/np.sum(psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_psd)
    
    return spectral_entropy

In [ ]:
# this function needs to be reviewed
def compute_spectral_entropy_f0(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    f0_sequence, _, _, _ = extract_f0_sequence(audio_file)
    
    f0_psd = np.zeros(len(f0_sequence))
    
    for i,f0 in enumerate(f0_sequence):
        if np.isnan(f0):
            f0_psd[i]=0
        else:
            difference_array = np.abs(freqs-f0)
            index = difference_array.argmin()
            f0_psd[i] = psd[index]
    
    normalized_f0_psd = f0_psd/np.sum(f0_psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_f0_psd)
    
    return spectral_entropy

In [ ]:
compute_spectral_entropy(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy_f0(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy(timit_real), compute_spectral_entropy(timit_fake)

In [ ]:
compute_spectral_entropy_f0(timit_real), compute_spectral_entropy(timit_fake)

## Testing for LibriSpeech and generated versions:

In [ ]:
LJ_original = '/Users/gautham/datasets/wavefake_data/LJSpeech_1.1/wavs'

LJ_fbmelgan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_full_band_melgan'

LJ_parallel_wavegan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_parallel_wavegan'

In [ ]:
original_files = []
for dirpath,_,filenames in os.walk(LJ_original):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            original_files.append(os.path.abspath(os.path.join(dirpath, file)))
original_files.sort()
print(len(original_files))

In [ ]:
melgan_files = []
for dirpath,_,filenames in os.walk(LJ_fbmelgan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            melgan_files.append(os.path.abspath(os.path.join(dirpath, file)))
melgan_files.sort()
print(len(melgan_files))

In [ ]:
wavegan_files = []
for dirpath,_,filenames in os.walk(LJ_parallel_wavegan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            wavegan_files.append(os.path.abspath(os.path.join(dirpath, file)))
wavegan_files.sort()
print(len(wavegan_files))

In [ ]:
compute_spectral_entropy(original_files[0]), compute_spectral_entropy(melgan_files[0]), compute_spectral_entropy(wavegan_files[0])

In [ ]:
compute_spectral_entropy_f0(original_files[0]), compute_spectral_entropy_f0(melgan_files[0]), compute_spectral_entropy_f0(wavegan_files[0])

In [ ]:
original_files_entropy = [compute_spectral_entropy(file) for file in original_files]
melgan_files_entropy = [compute_spectral_entropy(file) for file in melgan_files]
wavegan_files_entropy = [compute_spectral_entropy(file) for file in wavegan_files]

In [ ]:
plt.figure(figsize=(10, 10), dpi=80)
sns.kdeplot(original_files_entropy, label='original')
sns.kdeplot(melgan_files_entropy, label='melgan')
sns.kdeplot(wavegan_files_entropy, label='wavegan')
plt.legend()
plt.show()

In [ ]:
f0_orig, times_orig, _, _ = extract_f0_sequence(original_files[0]) 
f0_melgan, times_melgan, _, _ = extract_f0_sequence(melgan_files[0])
f0_wavegan, times_wavegan, _, _ = extract_f0_sequence(wavegan_files[0])

In [ ]:
print(original_files[0])
print(melgan_files[0])
print(wavegan_files[0])

In [ ]:
plot_f0_spectrogram(original_files[0], f0_orig, times_orig)

In [ ]:
plot_f0_spectrogram(melgan_files[0], f0_melgan, times_melgan)

In [ ]:
plot_f0_spectrogram(wavegan_files[0], f0_wavegan, times_wavegan)

In [ ]:
audio_ts, sr = librosa.load(original_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(melgan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(wavegan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

# Spectral Entropy

In [ ]:
import librosa
import os
import numpy as np
import scipy
from scipy import signal
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import sys
import pathlib
import glob
import seaborn as sns
plt.rcParams.update({'font.size': 18})

## Load Data

In [ ]:
base_path = "/home/ubuntu/"

biden_ai = base_path + 'data/world-leaders-dataset/biden_ai.wav'
biden_real = base_path + 'data/world-leaders-dataset/biden_original.wav'

timit_real = base_path + 'data/timit-test/example_real_resampled.wav'
timit_og_real = base_path + 'data/timit-test/example_real_original.wav'
timit_fake = base_path + 'data/timit-test/example_fake_resampled.wav'

## Extract F0

Reference: https://librosa.org/doc/main/generated/librosa.pyin.html

- __f0:__ time series of fundamental frequencies in Hertz

- __voiced_flag:__ time series containing boolean flags indicating whether a frame is voiced or not.

- __voiced_prob:__ time series containing the probability that a frame is voiced.

The min and max frequencies passed as params while extracting F0 recommended by librosa are 'C2' ~65 hz and 'C7' ~2093 hz.

The voiced speech of a typical adult male will have a fundamental frequency from 85 to 155 Hz, and that of a typical adult female from 165 to 255 Hz. [Voice frequency](https://en.wikipedia.org/wiki/Voice_frequency). 

Function returns `nan` values whenever there is no voiced segment


In [ ]:
def extract_f0_sequence(audio_file, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7')):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #use pYIN to extract f0 and voiced segment flag + probaility
    f0, voiced_flag, voiced_probs = librosa.pyin(audio, fmin=fmin, 
                                                 fmax=fmax, fill_na=np.nan)
    
    #generate ndarray of times (in seconds) corresponding to each frame of X
    times = librosa.times_like(f0)
    
    return f0, times, voiced_flag, voiced_probs

In [ ]:
def plot_f0_spectrogram(audio_file, f0_sequence, f0_times):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #generate spectrogram
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)
    
    #plot spectrogram from stft
    fig, ax = plt.subplots(figsize=(14, 5))
    img = librosa.display.specshow(D, x_axis='time', y_axis='log', ax=ax)
    ax.set(title='pYIN fundamental frequency estimation')
    fig.colorbar(img, ax=ax, format="%+2.f dB")
    
    #overlay f0 sequence
    ax.plot(f0_times, f0_sequence, label='f0', color='cyan', linewidth=3)
    ax.legend(loc='upper right')

In [ ]:
f0_jbreal, times_jb_real, _, _ = extract_f0_sequence(biden_real) 
f0_jbfake, times_jb_fake, _, _ = extract_f0_sequence(biden_ai) 

In [ ]:
plot_f0_spectrogram(biden_real, f0_jbreal, times_jb_real)

In [ ]:
plot_f0_spectrogram(biden_ai, f0_jbfake, times_jb_fake)

In [ ]:
f0_timit_real, times_timit_real, _, _ = extract_f0_sequence(timit_real)
f0_timit_og_real, times_timit_og_real, _, _ = extract_f0_sequence(timit_og_real) 
f0_timit_fake, times_timit_fake, _, _ = extract_f0_sequence(timit_fake)

In [ ]:
plot_f0_spectrogram(timit_real, f0_timit_real, times_timit_real)

In [ ]:
plot_f0_spectrogram(timit_og_real, f0_timit_og_real, times_timit_og_real)

In [ ]:
plot_f0_spectrogram(timit_fake, f0_timit_fake, times_timit_fake)

In [ ]:
audio_ts, sr = librosa.load(timit_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-6])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(timit_fake)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

## Spectral Entropy:

References:
- https://kilthub.cmu.edu/articles/thesis/Audio_Deepfake_Detection_Based_on_Differences_in_Human_and_Machine_Generated_Speech/21842454

- https://dsp.stackexchange.com/questions/23689/what-is-spectral-entropy

![image-2.png](attachment:image-2.png)


__Notes:__

- Typically PSD is computed using a periodogram. Is it OK to use an STFT for the audio signal?
- stft output is `freq_bins * frames`

In [ ]:
audio_ts, sr = librosa.load(biden_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(biden_ai)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
def compute_spectral_entropy(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    normalized_psd = psd/np.sum(psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_psd)
    
    return spectral_entropy

In [ ]:
# this function needs to be reviewed
def compute_spectral_entropy_f0(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    f0_sequence, _, _, _ = extract_f0_sequence(audio_file)
    
    f0_psd = np.zeros(len(f0_sequence))
    
    for i,f0 in enumerate(f0_sequence):
        if np.isnan(f0):
            f0_psd[i]=0
        else:
            difference_array = np.abs(freqs-f0)
            index = difference_array.argmin()
            f0_psd[i] = psd[index]
    
    normalized_f0_psd = f0_psd/np.sum(f0_psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_f0_psd)
    
    return spectral_entropy

In [ ]:
compute_spectral_entropy(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy_f0(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy(timit_real), compute_spectral_entropy(timit_fake)

In [ ]:
compute_spectral_entropy_f0(timit_real), compute_spectral_entropy(timit_fake)

## Testing for LibriSpeech and generated versions:

In [ ]:
LJ_original = '/Users/gautham/datasets/wavefake_data/LJSpeech_1.1/wavs'

LJ_fbmelgan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_full_band_melgan'

LJ_parallel_wavegan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_parallel_wavegan'

In [ ]:
original_files = []
for dirpath,_,filenames in os.walk(LJ_original):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            original_files.append(os.path.abspath(os.path.join(dirpath, file)))
original_files.sort()
print(len(original_files))

In [ ]:
melgan_files = []
for dirpath,_,filenames in os.walk(LJ_fbmelgan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            melgan_files.append(os.path.abspath(os.path.join(dirpath, file)))
melgan_files.sort()
print(len(melgan_files))

In [ ]:
wavegan_files = []
for dirpath,_,filenames in os.walk(LJ_parallel_wavegan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            wavegan_files.append(os.path.abspath(os.path.join(dirpath, file)))
wavegan_files.sort()
print(len(wavegan_files))

In [ ]:
compute_spectral_entropy(original_files[0]), compute_spectral_entropy(melgan_files[0]), compute_spectral_entropy(wavegan_files[0])

In [ ]:
compute_spectral_entropy_f0(original_files[0]), compute_spectral_entropy_f0(melgan_files[0]), compute_spectral_entropy_f0(wavegan_files[0])

In [ ]:
original_files_entropy = [compute_spectral_entropy(file) for file in original_files]
melgan_files_entropy = [compute_spectral_entropy(file) for file in melgan_files]
wavegan_files_entropy = [compute_spectral_entropy(file) for file in wavegan_files]

In [ ]:
plt.figure(figsize=(10, 10), dpi=80)
sns.kdeplot(original_files_entropy, label='original')
sns.kdeplot(melgan_files_entropy, label='melgan')
sns.kdeplot(wavegan_files_entropy, label='wavegan')
plt.legend()
plt.show()

In [ ]:
f0_orig, times_orig, _, _ = extract_f0_sequence(original_files[0]) 
f0_melgan, times_melgan, _, _ = extract_f0_sequence(melgan_files[0])
f0_wavegan, times_wavegan, _, _ = extract_f0_sequence(wavegan_files[0])

In [ ]:
print(original_files[0])
print(melgan_files[0])
print(wavegan_files[0])

In [ ]:
plot_f0_spectrogram(original_files[0], f0_orig, times_orig)

In [ ]:
plot_f0_spectrogram(melgan_files[0], f0_melgan, times_melgan)

In [ ]:
plot_f0_spectrogram(wavegan_files[0], f0_wavegan, times_wavegan)

In [ ]:
audio_ts, sr = librosa.load(original_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(melgan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(wavegan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

# Spectral Entropy

In [ ]:
import librosa
import os
import numpy as np
import scipy
from scipy import signal
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import sys
import pathlib
import glob
import seaborn as sns
plt.rcParams.update({'font.size': 18})

## Load Data

In [ ]:
base_path = "/home/ubuntu/"

biden_ai = base_path + 'data/world-leaders-dataset/biden_ai.wav'
biden_real = base_path + 'data/world-leaders-dataset/biden_original.wav'

timit_real = base_path + 'data/timit-test/example_real_resampled.wav'
timit_og_real = base_path + 'data/timit-test/example_real_original.wav'
timit_fake = base_path + 'data/timit-test/example_fake_resampled.wav'

## Extract F0

Reference: https://librosa.org/doc/main/generated/librosa.pyin.html

- __f0:__ time series of fundamental frequencies in Hertz

- __voiced_flag:__ time series containing boolean flags indicating whether a frame is voiced or not.

- __voiced_prob:__ time series containing the probability that a frame is voiced.

The min and max frequencies passed as params while extracting F0 recommended by librosa are 'C2' ~65 hz and 'C7' ~2093 hz.

The voiced speech of a typical adult male will have a fundamental frequency from 85 to 155 Hz, and that of a typical adult female from 165 to 255 Hz. [Voice frequency](https://en.wikipedia.org/wiki/Voice_frequency). 

Function returns `nan` values whenever there is no voiced segment


In [ ]:
def extract_f0_sequence(audio_file, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7')):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #use pYIN to extract f0 and voiced segment flag + probaility
    f0, voiced_flag, voiced_probs = librosa.pyin(audio, fmin=fmin, 
                                                 fmax=fmax, fill_na=np.nan)
    
    #generate ndarray of times (in seconds) corresponding to each frame of X
    times = librosa.times_like(f0)
    
    return f0, times, voiced_flag, voiced_probs

In [ ]:
def plot_f0_spectrogram(audio_file, f0_sequence, f0_times):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #generate spectrogram
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)
    
    #plot spectrogram from stft
    fig, ax = plt.subplots(figsize=(14, 5))
    img = librosa.display.specshow(D, x_axis='time', y_axis='log', ax=ax)
    ax.set(title='pYIN fundamental frequency estimation')
    fig.colorbar(img, ax=ax, format="%+2.f dB")
    
    #overlay f0 sequence
    ax.plot(f0_times, f0_sequence, label='f0', color='cyan', linewidth=3)
    ax.legend(loc='upper right')

In [ ]:
f0_jbreal, times_jb_real, _, _ = extract_f0_sequence(biden_real) 
f0_jbfake, times_jb_fake, _, _ = extract_f0_sequence(biden_ai) 

In [ ]:
plot_f0_spectrogram(biden_real, f0_jbreal, times_jb_real)

In [ ]:
plot_f0_spectrogram(biden_ai, f0_jbfake, times_jb_fake)

In [ ]:
f0_timit_real, times_timit_real, _, _ = extract_f0_sequence(timit_real)
f0_timit_og_real, times_timit_og_real, _, _ = extract_f0_sequence(timit_og_real) 
f0_timit_fake, times_timit_fake, _, _ = extract_f0_sequence(timit_fake)

In [ ]:
plot_f0_spectrogram(timit_real, f0_timit_real, times_timit_real)

In [ ]:
plot_f0_spectrogram(timit_og_real, f0_timit_og_real, times_timit_og_real)

In [ ]:
plot_f0_spectrogram(timit_fake, f0_timit_fake, times_timit_fake)

In [ ]:
audio_ts, sr = librosa.load(timit_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-6])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(timit_fake)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

## Spectral Entropy:

References:
- https://kilthub.cmu.edu/articles/thesis/Audio_Deepfake_Detection_Based_on_Differences_in_Human_and_Machine_Generated_Speech/21842454

- https://dsp.stackexchange.com/questions/23689/what-is-spectral-entropy

![image-2.png](attachment:image-2.png)


__Notes:__

- Typically PSD is computed using a periodogram. Is it OK to use an STFT for the audio signal?
- stft output is `freq_bins * frames`

In [ ]:
audio_ts, sr = librosa.load(biden_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(biden_ai)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
def compute_spectral_entropy(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    normalized_psd = psd/np.sum(psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_psd)
    
    return spectral_entropy

In [ ]:
# this function needs to be reviewed
def compute_spectral_entropy_f0(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    f0_sequence, _, _, _ = extract_f0_sequence(audio_file)
    
    f0_psd = np.zeros(len(f0_sequence))
    
    for i,f0 in enumerate(f0_sequence):
        if np.isnan(f0):
            f0_psd[i]=0
        else:
            difference_array = np.abs(freqs-f0)
            index = difference_array.argmin()
            f0_psd[i] = psd[index]
    
    normalized_f0_psd = f0_psd/np.sum(f0_psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_f0_psd)
    
    return spectral_entropy

In [ ]:
compute_spectral_entropy(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy_f0(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy(timit_real), compute_spectral_entropy(timit_fake)

In [ ]:
compute_spectral_entropy_f0(timit_real), compute_spectral_entropy(timit_fake)

## Testing for LibriSpeech and generated versions:

In [ ]:
LJ_original = '/Users/gautham/datasets/wavefake_data/LJSpeech_1.1/wavs'

LJ_fbmelgan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_full_band_melgan'

LJ_parallel_wavegan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_parallel_wavegan'

In [ ]:
original_files = []
for dirpath,_,filenames in os.walk(LJ_original):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            original_files.append(os.path.abspath(os.path.join(dirpath, file)))
original_files.sort()
print(len(original_files))

In [ ]:
melgan_files = []
for dirpath,_,filenames in os.walk(LJ_fbmelgan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            melgan_files.append(os.path.abspath(os.path.join(dirpath, file)))
melgan_files.sort()
print(len(melgan_files))

In [ ]:
wavegan_files = []
for dirpath,_,filenames in os.walk(LJ_parallel_wavegan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            wavegan_files.append(os.path.abspath(os.path.join(dirpath, file)))
wavegan_files.sort()
print(len(wavegan_files))

In [ ]:
compute_spectral_entropy(original_files[0]), compute_spectral_entropy(melgan_files[0]), compute_spectral_entropy(wavegan_files[0])

In [ ]:
compute_spectral_entropy_f0(original_files[0]), compute_spectral_entropy_f0(melgan_files[0]), compute_spectral_entropy_f0(wavegan_files[0])

In [ ]:
original_files_entropy = [compute_spectral_entropy(file) for file in original_files]
melgan_files_entropy = [compute_spectral_entropy(file) for file in melgan_files]
wavegan_files_entropy = [compute_spectral_entropy(file) for file in wavegan_files]

In [ ]:
plt.figure(figsize=(10, 10), dpi=80)
sns.kdeplot(original_files_entropy, label='original')
sns.kdeplot(melgan_files_entropy, label='melgan')
sns.kdeplot(wavegan_files_entropy, label='wavegan')
plt.legend()
plt.show()

In [ ]:
f0_orig, times_orig, _, _ = extract_f0_sequence(original_files[0]) 
f0_melgan, times_melgan, _, _ = extract_f0_sequence(melgan_files[0])
f0_wavegan, times_wavegan, _, _ = extract_f0_sequence(wavegan_files[0])

In [ ]:
print(original_files[0])
print(melgan_files[0])
print(wavegan_files[0])

In [ ]:
plot_f0_spectrogram(original_files[0], f0_orig, times_orig)

In [ ]:
plot_f0_spectrogram(melgan_files[0], f0_melgan, times_melgan)

In [ ]:
plot_f0_spectrogram(wavegan_files[0], f0_wavegan, times_wavegan)

In [ ]:
audio_ts, sr = librosa.load(original_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(melgan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(wavegan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

# Spectral Entropy

In [ ]:
import librosa
import os
import numpy as np
import scipy
from scipy import signal
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import sys
import pathlib
import glob
import seaborn as sns
plt.rcParams.update({'font.size': 18})

## Load Data

In [ ]:
base_path = "/home/ubuntu/"

biden_ai = base_path + 'data/world-leaders-dataset/biden_ai.wav'
biden_real = base_path + 'data/world-leaders-dataset/biden_original.wav'

timit_real = base_path + 'data/timit-test/example_real_resampled.wav'
timit_og_real = base_path + 'data/timit-test/example_real_original.wav'
timit_fake = base_path + 'data/timit-test/example_fake_resampled.wav'

## Extract F0

Reference: https://librosa.org/doc/main/generated/librosa.pyin.html

- __f0:__ time series of fundamental frequencies in Hertz

- __voiced_flag:__ time series containing boolean flags indicating whether a frame is voiced or not.

- __voiced_prob:__ time series containing the probability that a frame is voiced.

The min and max frequencies passed as params while extracting F0 recommended by librosa are 'C2' ~65 hz and 'C7' ~2093 hz.

The voiced speech of a typical adult male will have a fundamental frequency from 85 to 155 Hz, and that of a typical adult female from 165 to 255 Hz. [Voice frequency](https://en.wikipedia.org/wiki/Voice_frequency). 

Function returns `nan` values whenever there is no voiced segment


In [ ]:
def extract_f0_sequence(audio_file, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7')):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #use pYIN to extract f0 and voiced segment flag + probaility
    f0, voiced_flag, voiced_probs = librosa.pyin(audio, fmin=fmin, 
                                                 fmax=fmax, fill_na=np.nan)
    
    #generate ndarray of times (in seconds) corresponding to each frame of X
    times = librosa.times_like(f0)
    
    return f0, times, voiced_flag, voiced_probs

In [ ]:
def plot_f0_spectrogram(audio_file, f0_sequence, f0_times):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #generate spectrogram
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)
    
    #plot spectrogram from stft
    fig, ax = plt.subplots(figsize=(14, 5))
    img = librosa.display.specshow(D, x_axis='time', y_axis='log', ax=ax)
    ax.set(title='pYIN fundamental frequency estimation')
    fig.colorbar(img, ax=ax, format="%+2.f dB")
    
    #overlay f0 sequence
    ax.plot(f0_times, f0_sequence, label='f0', color='cyan', linewidth=3)
    ax.legend(loc='upper right')

In [ ]:
f0_jbreal, times_jb_real, _, _ = extract_f0_sequence(biden_real) 
f0_jbfake, times_jb_fake, _, _ = extract_f0_sequence(biden_ai) 

In [ ]:
plot_f0_spectrogram(biden_real, f0_jbreal, times_jb_real)

In [ ]:
plot_f0_spectrogram(biden_ai, f0_jbfake, times_jb_fake)

In [ ]:
f0_timit_real, times_timit_real, _, _ = extract_f0_sequence(timit_real)
f0_timit_og_real, times_timit_og_real, _, _ = extract_f0_sequence(timit_og_real) 
f0_timit_fake, times_timit_fake, _, _ = extract_f0_sequence(timit_fake)

In [ ]:
plot_f0_spectrogram(timit_real, f0_timit_real, times_timit_real)

In [ ]:
plot_f0_spectrogram(timit_og_real, f0_timit_og_real, times_timit_og_real)

In [ ]:
plot_f0_spectrogram(timit_fake, f0_timit_fake, times_timit_fake)

In [ ]:
audio_ts, sr = librosa.load(timit_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-6])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(timit_fake)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

## Spectral Entropy:

References:
- https://kilthub.cmu.edu/articles/thesis/Audio_Deepfake_Detection_Based_on_Differences_in_Human_and_Machine_Generated_Speech/21842454

- https://dsp.stackexchange.com/questions/23689/what-is-spectral-entropy

![image-2.png](attachment:image-2.png)


__Notes:__

- Typically PSD is computed using a periodogram. Is it OK to use an STFT for the audio signal?
- stft output is `freq_bins * frames`

In [ ]:
audio_ts, sr = librosa.load(biden_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(biden_ai)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
def compute_spectral_entropy(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    normalized_psd = psd/np.sum(psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_psd)
    
    return spectral_entropy

In [ ]:
# this function needs to be reviewed
def compute_spectral_entropy_f0(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    f0_sequence, _, _, _ = extract_f0_sequence(audio_file)
    
    f0_psd = np.zeros(len(f0_sequence))
    
    for i,f0 in enumerate(f0_sequence):
        if np.isnan(f0):
            f0_psd[i]=0
        else:
            difference_array = np.abs(freqs-f0)
            index = difference_array.argmin()
            f0_psd[i] = psd[index]
    
    normalized_f0_psd = f0_psd/np.sum(f0_psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_f0_psd)
    
    return spectral_entropy

In [ ]:
compute_spectral_entropy(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy_f0(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy(timit_real), compute_spectral_entropy(timit_fake)

In [ ]:
compute_spectral_entropy_f0(timit_real), compute_spectral_entropy(timit_fake)

## Testing for LibriSpeech and generated versions:

In [ ]:
LJ_original = '/Users/gautham/datasets/wavefake_data/LJSpeech_1.1/wavs'

LJ_fbmelgan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_full_band_melgan'

LJ_parallel_wavegan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_parallel_wavegan'

In [ ]:
original_files = []
for dirpath,_,filenames in os.walk(LJ_original):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            original_files.append(os.path.abspath(os.path.join(dirpath, file)))
original_files.sort()
print(len(original_files))

In [ ]:
melgan_files = []
for dirpath,_,filenames in os.walk(LJ_fbmelgan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            melgan_files.append(os.path.abspath(os.path.join(dirpath, file)))
melgan_files.sort()
print(len(melgan_files))

In [ ]:
wavegan_files = []
for dirpath,_,filenames in os.walk(LJ_parallel_wavegan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            wavegan_files.append(os.path.abspath(os.path.join(dirpath, file)))
wavegan_files.sort()
print(len(wavegan_files))

In [ ]:
compute_spectral_entropy(original_files[0]), compute_spectral_entropy(melgan_files[0]), compute_spectral_entropy(wavegan_files[0])

In [ ]:
compute_spectral_entropy_f0(original_files[0]), compute_spectral_entropy_f0(melgan_files[0]), compute_spectral_entropy_f0(wavegan_files[0])

In [ ]:
original_files_entropy = [compute_spectral_entropy(file) for file in original_files]
melgan_files_entropy = [compute_spectral_entropy(file) for file in melgan_files]
wavegan_files_entropy = [compute_spectral_entropy(file) for file in wavegan_files]

In [ ]:
plt.figure(figsize=(10, 10), dpi=80)
sns.kdeplot(original_files_entropy, label='original')
sns.kdeplot(melgan_files_entropy, label='melgan')
sns.kdeplot(wavegan_files_entropy, label='wavegan')
plt.legend()
plt.show()

In [ ]:
f0_orig, times_orig, _, _ = extract_f0_sequence(original_files[0]) 
f0_melgan, times_melgan, _, _ = extract_f0_sequence(melgan_files[0])
f0_wavegan, times_wavegan, _, _ = extract_f0_sequence(wavegan_files[0])

In [ ]:
print(original_files[0])
print(melgan_files[0])
print(wavegan_files[0])

In [ ]:
plot_f0_spectrogram(original_files[0], f0_orig, times_orig)

In [ ]:
plot_f0_spectrogram(melgan_files[0], f0_melgan, times_melgan)

In [ ]:
plot_f0_spectrogram(wavegan_files[0], f0_wavegan, times_wavegan)

In [ ]:
audio_ts, sr = librosa.load(original_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(melgan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(wavegan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

# Spectral Entropy

In [ ]:
import librosa
import os
import numpy as np
import scipy
from scipy import signal
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import sys
import pathlib
import glob
import seaborn as sns
plt.rcParams.update({'font.size': 18})

## Load Data

In [ ]:
base_path = "/home/ubuntu/"

biden_ai = base_path + 'data/world-leaders-dataset/biden_ai.wav'
biden_real = base_path + 'data/world-leaders-dataset/biden_original.wav'

timit_real = base_path + 'data/timit-test/example_real_resampled.wav'
timit_og_real = base_path + 'data/timit-test/example_real_original.wav'
timit_fake = base_path + 'data/timit-test/example_fake_resampled.wav'

## Extract F0

Reference: https://librosa.org/doc/main/generated/librosa.pyin.html

- __f0:__ time series of fundamental frequencies in Hertz

- __voiced_flag:__ time series containing boolean flags indicating whether a frame is voiced or not.

- __voiced_prob:__ time series containing the probability that a frame is voiced.

The min and max frequencies passed as params while extracting F0 recommended by librosa are 'C2' ~65 hz and 'C7' ~2093 hz.

The voiced speech of a typical adult male will have a fundamental frequency from 85 to 155 Hz, and that of a typical adult female from 165 to 255 Hz. [Voice frequency](https://en.wikipedia.org/wiki/Voice_frequency). 

Function returns `nan` values whenever there is no voiced segment


In [ ]:
def extract_f0_sequence(audio_file, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7')):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #use pYIN to extract f0 and voiced segment flag + probaility
    f0, voiced_flag, voiced_probs = librosa.pyin(audio, fmin=fmin, 
                                                 fmax=fmax, fill_na=np.nan)
    
    #generate ndarray of times (in seconds) corresponding to each frame of X
    times = librosa.times_like(f0)
    
    return f0, times, voiced_flag, voiced_probs

In [ ]:
def plot_f0_spectrogram(audio_file, f0_sequence, f0_times):
    
    #load the audio file
    audio, sr = librosa.load(audio_file)
    
    #generate spectrogram
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)
    
    #plot spectrogram from stft
    fig, ax = plt.subplots(figsize=(14, 5))
    img = librosa.display.specshow(D, x_axis='time', y_axis='log', ax=ax)
    ax.set(title='pYIN fundamental frequency estimation')
    fig.colorbar(img, ax=ax, format="%+2.f dB")
    
    #overlay f0 sequence
    ax.plot(f0_times, f0_sequence, label='f0', color='cyan', linewidth=3)
    ax.legend(loc='upper right')

In [ ]:
f0_jbreal, times_jb_real, _, _ = extract_f0_sequence(biden_real) 
f0_jbfake, times_jb_fake, _, _ = extract_f0_sequence(biden_ai) 

In [ ]:
plot_f0_spectrogram(biden_real, f0_jbreal, times_jb_real)

In [ ]:
plot_f0_spectrogram(biden_ai, f0_jbfake, times_jb_fake)

In [ ]:
f0_timit_real, times_timit_real, _, _ = extract_f0_sequence(timit_real)
f0_timit_og_real, times_timit_og_real, _, _ = extract_f0_sequence(timit_og_real) 
f0_timit_fake, times_timit_fake, _, _ = extract_f0_sequence(timit_fake)

In [ ]:
plot_f0_spectrogram(timit_real, f0_timit_real, times_timit_real)

In [ ]:
plot_f0_spectrogram(timit_og_real, f0_timit_og_real, times_timit_og_real)

In [ ]:
plot_f0_spectrogram(timit_fake, f0_timit_fake, times_timit_fake)

In [ ]:
audio_ts, sr = librosa.load(timit_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-6])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(timit_fake)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

## Spectral Entropy:

References:
- https://kilthub.cmu.edu/articles/thesis/Audio_Deepfake_Detection_Based_on_Differences_in_Human_and_Machine_Generated_Speech/21842454

- https://dsp.stackexchange.com/questions/23689/what-is-spectral-entropy

![image-2.png](attachment:image-2.png)


__Notes:__

- Typically PSD is computed using a periodogram. Is it OK to use an STFT for the audio signal?
- stft output is `freq_bins * frames`

In [ ]:
audio_ts, sr = librosa.load(biden_real)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(biden_ai)
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-7])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
def compute_spectral_entropy(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    normalized_psd = psd/np.sum(psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_psd)
    
    return spectral_entropy

In [ ]:
# this function needs to be reviewed
def compute_spectral_entropy_f0(audio_file):
    
    audio_ts, sr = librosa.load(audio_file)
    
    freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
    
    f0_sequence, _, _, _ = extract_f0_sequence(audio_file)
    
    f0_psd = np.zeros(len(f0_sequence))
    
    for i,f0 in enumerate(f0_sequence):
        if np.isnan(f0):
            f0_psd[i]=0
        else:
            difference_array = np.abs(freqs-f0)
            index = difference_array.argmin()
            f0_psd[i] = psd[index]
    
    normalized_f0_psd = f0_psd/np.sum(f0_psd)
    
    spectral_entropy = scipy.stats.entropy(normalized_f0_psd)
    
    return spectral_entropy

In [ ]:
compute_spectral_entropy(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy_f0(biden_real), compute_spectral_entropy(biden_ai)

In [ ]:
compute_spectral_entropy(timit_real), compute_spectral_entropy(timit_fake)

In [ ]:
compute_spectral_entropy_f0(timit_real), compute_spectral_entropy(timit_fake)

## Testing for LibriSpeech and generated versions:

In [ ]:
LJ_original = '/Users/gautham/datasets/wavefake_data/LJSpeech_1.1/wavs'

LJ_fbmelgan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_full_band_melgan'

LJ_parallel_wavegan = '/Users/gautham/datasets/wavefake_data/generated_audio/ljspeech_parallel_wavegan'

In [ ]:
original_files = []
for dirpath,_,filenames in os.walk(LJ_original):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            original_files.append(os.path.abspath(os.path.join(dirpath, file)))
original_files.sort()
print(len(original_files))

In [ ]:
melgan_files = []
for dirpath,_,filenames in os.walk(LJ_fbmelgan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            melgan_files.append(os.path.abspath(os.path.join(dirpath, file)))
melgan_files.sort()
print(len(melgan_files))

In [ ]:
wavegan_files = []
for dirpath,_,filenames in os.walk(LJ_parallel_wavegan):
    for file in filenames:
        if file.startswith("LJ001-") and file.endswith('.wav'):
        #if file.endswith('.wav'):
            wavegan_files.append(os.path.abspath(os.path.join(dirpath, file)))
wavegan_files.sort()
print(len(wavegan_files))

In [ ]:
compute_spectral_entropy(original_files[0]), compute_spectral_entropy(melgan_files[0]), compute_spectral_entropy(wavegan_files[0])

In [ ]:
compute_spectral_entropy_f0(original_files[0]), compute_spectral_entropy_f0(melgan_files[0]), compute_spectral_entropy_f0(wavegan_files[0])

In [ ]:
original_files_entropy = [compute_spectral_entropy(file) for file in original_files]
melgan_files_entropy = [compute_spectral_entropy(file) for file in melgan_files]
wavegan_files_entropy = [compute_spectral_entropy(file) for file in wavegan_files]

In [ ]:
plt.figure(figsize=(10, 10), dpi=80)
sns.kdeplot(original_files_entropy, label='original')
sns.kdeplot(melgan_files_entropy, label='melgan')
sns.kdeplot(wavegan_files_entropy, label='wavegan')
plt.legend()
plt.show()

In [ ]:
f0_orig, times_orig, _, _ = extract_f0_sequence(original_files[0]) 
f0_melgan, times_melgan, _, _ = extract_f0_sequence(melgan_files[0])
f0_wavegan, times_wavegan, _, _ = extract_f0_sequence(wavegan_files[0])

In [ ]:
print(original_files[0])
print(melgan_files[0])
print(wavegan_files[0])

In [ ]:
plot_f0_spectrogram(original_files[0], f0_orig, times_orig)

In [ ]:
plot_f0_spectrogram(melgan_files[0], f0_melgan, times_melgan)

In [ ]:
plot_f0_spectrogram(wavegan_files[0], f0_wavegan, times_wavegan)

In [ ]:
audio_ts, sr = librosa.load(original_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(melgan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()

In [ ]:
audio_ts, sr = librosa.load(wavegan_files[0])
freqs, psd = scipy.signal.periodogram(audio_ts, sr, nfft=2048)
plt.figure(figsize=(16, 6), dpi=80)
plt.semilogy(freqs, psd)
plt.ylim([1e-18, 1e-3])
plt.xticks(np.arange(0,12000,1000))
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.show()